In [18]:
import pandas as pd
import os
import re

In [14]:
run_folders = ['../../output/run1/validation/summary-statistics', '../../output/run2/validation/summary-statistics', 
               '../../output/run3/validation/summary-statistics']
graph_types = ['summary_observation', 'summary_action']

In [21]:
all_dfs = []

method_pattern = re.compile(r'graph/([^/]+)/kg\.ttl')

for run_index, folder in enumerate(run_folders, start=1):
    for graph_type in graph_types:
        file_path = os.path.join(folder, f"{graph_type}.csv")
        df = pd.read_csv(file_path, index_col=False)
        first_column = df.columns[0]
        df.rename(columns={first_column: 'file_path'}, inplace=True)
        df['method'] = df['file_path'].astype(str).apply(
            lambda x: method_pattern.search(x).group(1) if method_pattern.search(x) else 'UNKNOWN'
        )
        df['run'] = run_index
        df['graph_type'] = graph_type.replace('summary_', '')
        columns = ['run', 'graph_type', 'method'] + [
            col for col in df.columns if col not in ['file_path', 'run', 'graph_type', 'method']
        ]
        df = df[columns]
        all_dfs.append(df)

merged_df = pd.concat(all_dfs, ignore_index=True)
percentage_columns = [
    col for col in merged_df.columns
    if merged_df[col].astype(str).str.contains('%').any()
]
for col in percentage_columns:
    merged_df[col] = (
        merged_df[col].astype(str)
        .str.replace('%', '', regex=False)
        .replace('', '0')
        .astype(float) / 100
    )
#merged_df.to_csv('merged_output.csv', index=False)
print(merged_df.head())

   run   graph_type    method  Full Parse OK  Total triples in KG  \
0    1  observation       dpe           True                   17   
1    1  observation      i2kg           True                   31   
2    1  observation      d2kg           True                   99   
3    1  observation  d2kg-rag           True                   89   
4    1  observation       dpe           True                   35   

   Valid triples  Invalid triples  Distinct classes used  Class Compliance  \
0             17                0                      0               0.0   
1             31                0                      5               0.0   
2             99                0                      6               1.0   
3             89                0                      6               1.0   
4             35                0                      0               0.0   

   Class Coverage  Distinct properties used  Property Compliance  \
0            0.00                         2     